# Convolution Nuclear Norm Minimization (ConvNNM)

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def conv(vec, kernel_size):
    n = vec.shape[0]
    mat = np.zeros((n, kernel_size))
    mat[:, 0] = vec
    for k in range(1, kernel_size):
        mat[:, k] = np.append(vec[n - k :], vec[: n - k], axis = 0)
    return mat

def inv_conv(mat):
    kernel_size = mat.shape[1]
    vec = mat[:, 0]
    for k in range(1, kernel_size):
        vec += np.append(mat[k :, k], mat[: k, k], axis = 0)
    return vec / kernel_size

def svt(mat, tau):
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    vec = s - tau
    vec[vec < 0] = 0
    return u @ np.diag(vec) @ v

def ConvNNM(y_true, y, kernel_size, lmbda, maxiter = 50):
    eta = 100
    n = y.shape[0]
    pos_train = np.where(y != 0)
    pos_test = np.where((y_true != 0) & (y == 0))
    z = y.copy()
    Z = conv(z, kernel_size)
    W = np.zeros((n, kernel_size))
    show_iter = 10
    for it in range(maxiter):
        X = svt(Z - W * kernel_size / lmbda, kernel_size / lmbda)
        z = inv_conv(X + W * kernel_size / lmbda)
        z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train]
                        + eta / (lmbda + eta) * y[pos_train])
        Z = conv(z, kernel_size)
        W = W + lmbda / kernel_size * (X - Z)
        y_hat = inv_conv(X)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_true[pos_test], y_hat[pos_test]))
            print(compute_rmse(y_true[pos_test], y_hat[pos_test]))
            print()
    return y_hat

## Portland Freeway Traffic Speed Data

In [ ]:
import numpy as np
np.random.seed(1)
import time

missing_rate = 0.95
print('Missing rate = {}'.format(missing_rate))

dense_mat = np.load('../datasets/Portland-data-set/speed.npy')
d = 3
dense_vec = dense_mat[0, : 96 * d]
T = dense_vec.shape[0]
sparse_vec = dense_vec * np.round(np.random.rand(T) + 0.5 - missing_rate)

kernel_size = 48
lmbda = 5e-3 * kernel_size
maxiter = 100
x = ConvNNM(dense_vec, sparse_vec, kernel_size, lmbda, maxiter)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 13})

fig = plt.figure(figsize = (7.5, 2.2))
ax = fig.add_subplot(111)
plt.plot(dense_vec[: 96 * d], 'dodgerblue', linewidth = 1)
plt.plot(x[: 96 * d], 'red', linewidth = 2.5)
plt.plot(np.arange(0, 96 * d), sparse_vec[: 96 * d], 'o', 
         markeredgecolor = 'darkblue', 
         markerfacecolor = 'deepskyblue', markersize = 10)
plt.xlabel('Time')
plt.ylabel('Speed (mph)')
plt.xlim([0, 96 * d])
plt.ylim([54, 65])
plt.xticks(np.arange(0, 96 * d + 1, 24))
plt.yticks(np.arange(54, 66, 2))
plt.grid(linestyle = '-.', linewidth = 0.5)
ax.tick_params(direction = 'in')

plt.savefig('speeds_cnnm.pdf', bbox_inches = "tight")
plt.show()

## Portland Freeway Traffic Volume Data

In [ ]:
import numpy as np
np.random.seed(1)
import time

missing_rate = 0.95
print('Missing rate = {}'.format(missing_rate))

dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
d = 3
dense_vec = dense_mat[0, : 96 * d]
T = dense_vec.shape[0]
sparse_vec = dense_vec * np.round(np.random.rand(T) + 0.5 - missing_rate)

kernel_size = 96 * 2
lmbda = 1e-4 * kernel_size
maxiter = 100
x = ConvNNM(dense_vec, sparse_vec, kernel_size, lmbda, maxiter)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})

fig = plt.figure(figsize = (5.5, 2.4))
ax = fig.add_subplot(111)
plt.plot(dense_vec[: 96 * d], 'dodgerblue', linewidth = 1.5)
plt.plot(x[: 96 * d], 'red', linewidth = 1.2)
pos = np.where(sparse_vec != 0)
plt.plot(pos[0], sparse_vec[pos], 'o', 
         markeredgecolor = 'darkblue', 
         markerfacecolor = 'deepskyblue', markersize = 10)
plt.xlabel('Time')
plt.ylabel('Volume')
plt.xticks(np.arange(0, 96 * d + 1, 48))
plt.xlim([0, 96 * d])
plt.yticks(np.arange(0, 301, 100))
plt.ylim([0, 300])
plt.grid(linestyle = '-.', linewidth = 0.5)
ax.tick_params(direction = 'in')

plt.savefig('volumes_cnnm.pdf', bbox_inches = "tight")
plt.show()

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>